# 분류

## KNN 알고리즘

In [ ]:
'''
KNN (K-Nearset-Keighbors) : K개의 가까운 이웃
    - 새로운 관측값이 주어지면 기존 데이터 중에서 가장 속성이 비슷한 K개의 이웃을 먼저 찾음
      그 가까운 이웃들이 갖고 있는 목표값과 같은 값으로 분류하여 예측하는 알고리즘

K값에 따라 예측의 정확도가 달라지므로, 적절한 K값을 찾는 것이 매우 중요하다.
'''

In [1]:
import pandas as pd
import seaborn as sns

In [2]:
# 데이터 로딩
df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [3]:
# 데이터 탐색
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [4]:
# 데이터 전처리
# NaN이 너무 많은 deck열, 중복되는 embark_town열 삭제
rdf = df.drop(['deck', 'embark_town'], axis=1)
rdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   survived    891 non-null    int64   
 1   pclass      891 non-null    int64   
 2   sex         891 non-null    object  
 3   age         714 non-null    float64 
 4   sibsp       891 non-null    int64   
 5   parch       891 non-null    int64   
 6   fare        891 non-null    float64 
 7   embarked    889 non-null    object  
 8   class       891 non-null    category
 9   who         891 non-null    object  
 10  adult_male  891 non-null    bool    
 11  alive       891 non-null    object  
 12  alone       891 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(4)
memory usage: 72.5+ KB


In [6]:
rdf.columns.values  #컬럼항목만 보기

array(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'alive', 'alone'],
      dtype=object)

In [7]:
# age열에서 NaN인 행을 삭제 (복습할 때는 평균값으로 치환해서 수행해보세요)
rdf = rdf.dropna(subset=['age'], how='any', axis=0)
len(rdf)  # dropna이후 714개

714

In [8]:
# embarked 열의 NaN값을 embarked에서 가장 많이 나온 도시값으로 치환
freq = rdf['embarked'].value_counts(dropna=True).idxmax()
freq  # S가 가장 많음

'S'

In [9]:
rdf.describe(include='all')  # 데이터 통계

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,alive,alone
count,714.000000,714.000000,714,714.000000,714.000000,714.000000,714.000000,712,714,714,714,714,714
unique,NaN,NaN,2,NaN,NaN,NaN,NaN,3,3,3,2,2,2
top,NaN,NaN,male,NaN,NaN,NaN,NaN,S,Third,man,True,no,True
freq,NaN,NaN,453,NaN,NaN,NaN,NaN,554,355,413,413,424,404
mean,0.406162,2.236695,NaN,29.699118,0.512605,0.431373,34.694514,NaN,NaN,NaN,NaN,NaN,NaN
std,0.491460,0.838250,NaN,14.526497,0.929783,0.853289,52.918930,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,1.000000,NaN,0.420000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,0.000000,1.000000,NaN,20.125000,0.000000,0.000000,8.050000,NaN,NaN,NaN,NaN,NaN,NaN
50%,0.000000,2.000000,NaN,28.000000,0.000000,0.000000,15.741700,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.000000,3.000000,NaN,38.000000,1.000000,1.000000,33.375000,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# 치환
rdf['embarked'].fillna(freq, inplace=True)

C:\Users\goott7\anaconda3\lib\site-packages\pandas\core\series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [11]:
rdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   survived    714 non-null    int64   
 1   pclass      714 non-null    int64   
 2   sex         714 non-null    object  
 3   age         714 non-null    float64 
 4   sibsp       714 non-null    int64   
 5   parch       714 non-null    int64   
 6   fare        714 non-null    float64 
 7   embarked    714 non-null    object  
 8   class       714 non-null    category
 9   who         714 non-null    object  
 10  adult_male  714 non-null    bool    
 11  alive       714 non-null    object  
 12  alone       714 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(4)
memory usage: 63.6+ KB


In [17]:
################
y = rdf['survived']
x = rdf.drop(['survived','sex','embarked','who','alive'], axis=1)

In [18]:
# 학습 세트와 테스트 데이터 세트 분리
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)

In [19]:
# KNN 분류 모형으로 학습/예측/평가
from sklearn.neighbors import KNeighborsClassifier

# K 이웃을 5로 임의 설정
knn = KNeighborsClassifier(n_neighbors=5)

# train data로 학습
knn.fit(X_train, y_train)

# test data로 클래스 예측 (분류)
y_pred = knn.predict(X_test)
y_pred[:10]
# x,y나누는거 임의라서 오류나는듯

C:\Users\goott7\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)


ValueError: Unable to convert array of bytes/strings into decimal numbers with dtype='numeric'

In [20]:
y_test[:10]

585    1
341    1
636    0
716    1
817    0
237    1
407    1
371    0
512    1
370    1
Name: survived, dtype: int64

In [21]:
# 모형 성능 평가 - 오차행렬 체크
from sklearn.metrics import confusion_matrix

knn_matrix = confusion_matrix(y_test, y_pred)
knn_matrix

NameError: name 'y_pred' is not defined

In [ ]:
# 평가 지표 계산
#from sklearn.



## SVM 알고리즘

In [ ]:
'''
SVM (Support Vector Machine)
 - 설명변수의 열 벡터는 각각의 고유의 축을 갖는 벡터 공간을 만들 수 있고, 분석 대상이 되는 개별 관측값은 모든 속성(열 벡터)에 
   관한 값을 해당 축의 좌표로 표시하여 벡터 공간에서의 위치로 나타내는 방법
 - 만약 속성이 2개 존재하는 데이터셋이라면 2차원 평면공각, 3개라면 3차원 공간에, 4개 이상이라면 고차원 벡터 공간 좌표를 사용
 
학습 방법 : 벡터 공간에 위치한 훈련 데이터의 좌표와 각 데이터가 어떤 분류값을 가져야 하는지 레이블을 입력받아서 학습하는 형태

'''

In [ ]:
########

In [ ]:
## Decision Tree (결정트리)

In [22]:
import pandas as pd
import numpy as np

In [26]:
# 데이터 로딩
uci_data = 'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data'

df = pd.read_csv(uci_data, header=None)  # data에 헤더가 없어서 옵션 추가
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [ ]:
'''
dataset describe
샘플id | 암세포 조직 크기와 모양 등 종양 특성 열 9개 | 악성 종양 여부 (2:양성, 4:악성)
'''

In [27]:
# 데이터 탐색
df.info()  # 6이 오브젝트임

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       699 non-null    int64 
 1   1       699 non-null    int64 
 2   2       699 non-null    int64 
 3   3       699 non-null    int64 
 4   4       699 non-null    int64 
 5   5       699 non-null    int64 
 6   6       699 non-null    object
 7   7       699 non-null    int64 
 8   8       699 non-null    int64 
 9   9       699 non-null    int64 
 10  10      699 non-null    int64 
dtypes: int64(10), object(1)
memory usage: 60.2+ KB


In [30]:
# 컬럼 이름 지정
df.columns = ['id','clump','cell_size','cell_shape','adhesion','epithlial',
              'bare_nucleoli','chromatin','normal_nucleoli','mitoses','class']

df.head(10)

,id,clump,cell_size,cell_shape,adhesion,epithlial,bare_nucleoli,chromatin,normal_nucleoli,mitoses,class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2
5,1017122,8,10,10,8,7,10,9,7,1,4
6,1018099,1,1,1,1,2,10,3,1,1,2
7,1018561,2,1,2,1,2,1,3,1,1,2
8,1033078,2,1,1,1,2,1,1,1,5,2
9,1033078,4,2,1,1,2,1,2,1,1,2


In [32]:
# 요약정보 확인
df.describe()  # bare_nucleoli 안나옴, 오브젝트라서 연산 불가

,id,clump,cell_size,cell_shape,adhesion,epithlial,chromatin,normal_nucleoli,mitoses,class
count,6.990000e+02,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000
mean,1.071704e+06,4.417740,3.134478,3.207439,2.806867,3.216023,3.437768,2.866953,1.589413,2.689557
std,6.170957e+05,2.815741,3.051459,2.971913,2.855379,2.214300,2.438364,3.053634,1.715078,0.951273
min,6.163400e+04,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000
25%,8.706885e+05,2.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,1.000000,2.000000
50%,1.171710e+06,4.000000,1.000000,1.000000,1.000000,2.000000,3.000000,1.000000,1.000000,2.000000
75%,1.238298e+06,6.000000,5.000000,5.000000,4.000000,4.000000,5.000000,4.000000,1.000000,4.000000
max,1.345435e+07,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000


In [33]:
# bare_nucleoli 열의 특이한 데이터를 확인
df['bare_nucleoli'].unique()  # '?'가 있음

array(['1', '10', '2', '4', '3', '9', '7', '?', '5', '8', '6'],
      dtype=object)

In [36]:
# 데이터 전처리
# ? -> NaN -> NaN삭제 -> object to int
df['bare_nucleoli'].replace('?', np.nan, inplace=True)  # NaN으로
df.dropna(subset=['bare_nucleoli'], axis=0, inplace=True) # NaN 존재행 삭제
df['bare_nucleoli'] = df['bare_nucleoli'].astype('int64')  #그냥 int하니까 int32들어감

In [37]:
df.info()  #16개 지워짐

<class 'pandas.core.frame.DataFrame'>
Int64Index: 683 entries, 0 to 698
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   id               683 non-null    int64
 1   clump            683 non-null    int64
 2   cell_size        683 non-null    int64
 3   cell_shape       683 non-null    int64
 4   adhesion         683 non-null    int64
 5   epithlial        683 non-null    int64
 6   bare_nucleoli    683 non-null    int64
 7   chromatin        683 non-null    int64
 8   normal_nucleoli  683 non-null    int64
 9   mitoses          683 non-null    int64
 10  class            683 non-null    int64
dtypes: int64(11)
memory usage: 64.0 KB


In [38]:
df.describe()

,id,clump,cell_size,cell_shape,adhesion,epithlial,bare_nucleoli,chromatin,normal_nucleoli,mitoses,class
count,6.830000e+02,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000
mean,1.076720e+06,4.442167,3.150805,3.215227,2.830161,3.234261,3.544656,3.445095,2.869693,1.603221,2.699854
std,6.206440e+05,2.820761,3.065145,2.988581,2.864562,2.223085,3.643857,2.449697,3.052666,1.732674,0.954592
min,6.337500e+04,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000
25%,8.776170e+05,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000
50%,1.171795e+06,4.000000,1.000000,1.000000,1.000000,2.000000,1.000000,3.000000,1.000000,1.000000,2.000000
75%,1.238705e+06,6.000000,5.000000,5.000000,4.000000,4.000000,6.000000,5.000000,4.000000,1.000000,4.000000
max,1.345435e+07,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000


In [40]:
# 설명 변수 X / 목적 변수 y 구분
X = df[['clump','cell_size','cell_shape','adhesion','epithlial',
              'bare_nucleoli','chromatin','normal_nucleoli','mitoses']]
### id 필요없고, class가 목적임
y = df['class']

In [43]:
# 정규화
from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit(X).transform(X)

# 훈련세트 / 데이터세트 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=10)

print('train data shape:', X_train.shape)
print('test data shape:', X_test.shape)

train data shape: (512, 9)
test data shape: (171, 9)


In [ ]:
'''
Decision Tree 분류 모형
구조 : 컴퓨터 알고리즘에서 사용하는 트리(tree) 구조를 사용
     : 각 분기점(node)에는 분석대상의 속성(설명 변수)들이 위치하고, 해당 속성이 갖는 값을 이용하여 새로운 가지(branch)를 만드는 방식
     : 각 분기점에서 최적의 속성을 선택할 때는 해당 속서을 기준으로 분류한 값들이 구분되는 정도를 측정
       => 다른 종류의 값들이 섞여있는 정도를 나타낼 때는 entropy를 주로 활용 - 낮은쪽으로 뻗어나감
       => 이 entropy가 낮을수록 분류가 잘 되는 형태
       => entropy가 일정 수준 이하로 낮아질 때 까지 과정을 반복
'''

In [46]:
from sklearn.tree import DecisionTreeClassifier

# max_depth : tree 레벨
#           -> 많이 주면 정확도가 높아지지만 과대적합 위험성 있음
tree_model = DecisionTreeClassifier(criterion='entropy', max_depth=5)

tree_model.fit(X_train, y_train)

y_pred = tree_model.predict(X_test)

print(y_pred[:10])
print('-----------------------------------------')
print(y_test.values[:10])

[4 4 4 4 4 4 2 2 4 4]
-----------------------------------------
[4 4 4 4 4 4 2 2 4 4]


In [47]:
# 평가
from sklearn import metrics

tree_matrix = metrics.confusion_matrix(y_test, y_pred)
tree_matrix
# 좌상 TN, 우하 TY?  

array([[106,   3],
       [  1,  61]], dtype=int64)

In [48]:
tree_report = metrics.classification_report(y_test, y_pred)
print(tree_report)

              precision    recall  f1-score   support

           2       0.99      0.97      0.98       109
           4       0.95      0.98      0.97        62

    accuracy                           0.98       171
   macro avg       0.97      0.98      0.97       171
weighted avg       0.98      0.98      0.98       171

